In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
train_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/H37Rv_MIC_train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/H37Rv_MIC_test.csv")
train_df.head(1)

,data_soucrce,value,unit,type,measurement,relation,smi,description,category,origin,rdkit_smi,mw,logp
0,ChEMBL_H37Rv,-3.79314,nM,In Vitro,pMIC,=,CSc1ccc(-c2nc3sc(S(N)(=O)=O)nn3c2-c2ccccc2)cc1,Antimycobacterial activity against Mycobacteri...,Tuberculosis,H37Rv,CSc1ccc(-c2nc3sc(S(N)(=O)=O)nn3c2-c2ccccc2)cc1,402.526,3.4941


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5899 entries, 0 to 5898
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   data_soucrce  5899 non-null   object 
 1   value         5899 non-null   float64
 2   unit          5899 non-null   object 
 3   type          5897 non-null   object 
 4   measurement   5899 non-null   object 
 5   relation      5899 non-null   object 
 6   smi           5899 non-null   object 
 7   description   5899 non-null   object 
 8   category      5899 non-null   object 
 9   origin        5899 non-null   object 
 10  rdkit_smi     5899 non-null   object 
 11  mw            5899 non-null   float64
 12  logp          5899 non-null   float64
dtypes: float64(3), object(10)
memory usage: 599.2+ KB


In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1475 entries, 0 to 1474
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   data_soucrce  1475 non-null   object 
 1   value         1475 non-null   float64
 2   unit          1475 non-null   object 
 3   type          1475 non-null   object 
 4   measurement   1475 non-null   object 
 5   relation      1475 non-null   object 
 6   smi           1475 non-null   object 
 7   description   1475 non-null   object 
 8   category      1475 non-null   object 
 9   origin        1475 non-null   object 
 10  rdkit_smi     1475 non-null   object 
 11  mw            1475 non-null   float64
 12  logp          1475 non-null   float64
dtypes: float64(3), object(10)
memory usage: 149.9+ KB


In [6]:
train_df = train_df.rename(columns={'data_soucrce': 'data_source'})
train_df.columns

Index(['data_source', 'value', 'unit', 'type', 'measurement', 'relation',
       'smi', 'description', 'category', 'origin', 'rdkit_smi', 'mw', 'logp'],
      dtype='object')

In [7]:
test_df = test_df.rename(columns={'data_soucrce': 'data_source'})
test_df.columns

Index(['data_source', 'value', 'unit', 'type', 'measurement', 'relation',
       'smi', 'description', 'category', 'origin', 'rdkit_smi', 'mw', 'logp'],
      dtype='object')

In [8]:
train_df.nunique()

data_source       3
value          4581
unit              1
type              1
measurement       1
relation          1
smi            5899
description     449
category          1
origin            1
rdkit_smi      5899
mw             5065
logp           5294
dtype: int64

In [9]:
test_df.nunique()

data_source       2
value          1302
unit              1
type              1
measurement       1
relation          1
smi            1475
description     346
category          1
origin            1
rdkit_smi      1475
mw             1421
logp           1442
dtype: int64

In [10]:
!pip install rdkit-pypi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 40.8 MB/s eta 0:00:00


In [11]:
# -*- coding: utf-8 -*-
"""
Created on Fri Dec 20 15:37:59 2019

@author: Gil

sd: list of rdkit Chem mol instance.
"""

from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys                       #MACCS Keys (166 bit)
from rdkit.Chem.Fingerprints import FingerprintMols    #Topological Fingerprints (RDKFingerprint, 2048 bit)
from rdkit.Avalon import pyAvalonTools                 #512 bit
from rdkit.Chem.AtomPairs import Pairs, Torsions       #Atom pair & torsions fingerprints

from rdkit.Chem.EState.Fingerprinter import FingerprintMol # https://www.rdkit.org/docs/source/rdkit.Chem.EState.Fingerprinter.html (79 bit)
from rdkit.Chem.EState import EState_VSA # https://www.rdkit.org/docs/source/rdkit.Chem.EState.EState_VSA.html
from rdkit.Chem import GraphDescriptors #https://www.rdkit.org/docs/source/rdkit.Chem.GraphDescriptors.html
from rdkit.Chem import Descriptors #https://www.rdkit.org/docs/source/rdkit.Chem.Descriptors.html
from rdkit.Chem import Crippen #https://www.rdkit.org/docs/source/rdkit.Chem.Crippen.html
from rdkit.Chem import MolSurf #https://www.rdkit.org/docs/source/rdkit.Chem.MolSurf.html
from rdkit.Chem import rdMolDescriptors #https://www.rdkit.org/docs/source/rdkit.Chem.rdMolDescriptors.html
from rdkit.Chem import Lipinski

from rdkit import Chem, DataStructs

import numpy as np
import pandas as pd

def fingerprint_bitgenerator(fps):
    np_fps = []

    for fp in fps:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fp,arr)
        np_fps.append(arr)

    return np_fps

def atompairfingerprint_keyextractor(fps):
    fpkeys = []

    for fp in fps:
        fpkeys += list(fp.GetNonzeroElements().keys())

    fpkeys = list(set(fpkeys))
    fpkeys.sort()
    return fpkeys

def atompair_bitgenerator(fps, keys):

    keylen = len(keys)
    np_fps = []
    for fp in fps:
        arr = np.zeros(keylen)
        for i in range(keylen):
            arr[i]=fp[keys[i]]
        np_fps.append(arr)

    return np_fps

def description2D_calc(sd):
    #Topliss fragments / BCUT2D
    desc_2d = []
    for m in sd:
        each_desc = []

        desc = [rdMolDescriptors.CalcNumAliphaticCarbocycles(m),
                rdMolDescriptors.CalcNumAliphaticHeterocycles(m),
                rdMolDescriptors.CalcNumAliphaticRings(m),
                rdMolDescriptors.CalcNumAmideBonds(m),
                rdMolDescriptors.CalcNumAromaticCarbocycles(m),
                rdMolDescriptors.CalcNumAromaticHeterocycles(m),
                rdMolDescriptors.CalcNumAromaticRings(m),
                rdMolDescriptors.CalcNumAtomStereoCenters(m),
                rdMolDescriptors.CalcNumBridgeheadAtoms(m),
                rdMolDescriptors.CalcNumHBA(m),
                rdMolDescriptors.CalcNumHBD(m),
                rdMolDescriptors.CalcNumHeteroatoms(m),
                rdMolDescriptors.CalcNumHeterocycles(m),
                rdMolDescriptors.CalcNumLipinskiHBA(m),
                rdMolDescriptors.CalcNumLipinskiHBD(m),
                rdMolDescriptors.CalcNumRings(m),
                rdMolDescriptors.CalcNumRotatableBonds(m),
                rdMolDescriptors.CalcNumSaturatedCarbocycles(m),
                rdMolDescriptors.CalcNumSaturatedHeterocycles(m),
                rdMolDescriptors.CalcNumSaturatedRings(m),
                rdMolDescriptors.CalcNumSpiroAtoms(m),

                Lipinski.FractionCSP3(m),
                Lipinski.HeavyAtomCount(m),
                Lipinski.NHOHCount(m),
                Lipinski.NOCount(m),

                Descriptors.ExactMolWt(m),
                Descriptors.HeavyAtomMolWt(m),
                Descriptors.MaxAbsEStateIndex(m),
                Descriptors.MaxEStateIndex(m),
                Descriptors.MinAbsEStateIndex(m),
                Descriptors.MinEStateIndex(m),
                Descriptors.MaxAbsPartialCharge(m),
                Descriptors.MaxPartialCharge(m),
                Descriptors.MinAbsPartialCharge(m),
                Descriptors.MinPartialCharge(m),
                Descriptors.NumRadicalElectrons(m),
                Descriptors.NumValenceElectrons(m),

                Crippen.MolLogP(m),
                Crippen.MolMR(m),

                MolSurf.LabuteASA(m),
                MolSurf.TPSA(m)]

        graph_desc = [GraphDescriptors.BalabanJ(m),
                      GraphDescriptors.BertzCT(m),
                      GraphDescriptors.Ipc(m),
                      GraphDescriptors.Chi0(m),
                      GraphDescriptors.Chi0n(m),
                      GraphDescriptors.Chi0v(m),
                      GraphDescriptors.Chi1(m),
                      GraphDescriptors.Chi1n(m),
                      GraphDescriptors.Chi1v(m),
                      GraphDescriptors.Chi2n(m),
                      GraphDescriptors.Chi2v(m),
                      GraphDescriptors.Chi3n(m),
                      GraphDescriptors.Chi3v(m),
                      GraphDescriptors.Chi4n(m),
                      GraphDescriptors.Chi4v(m),
                      GraphDescriptors.HallKierAlpha(m),
                      GraphDescriptors.Kappa1(m),
                      GraphDescriptors.Kappa2(m),
                      GraphDescriptors.Kappa3(m)]

        estate_vsa = list(EState_VSA.EState_VSA_(m))
        vsa_estate = list(EState_VSA.VSA_EState_(m))

        peoe_vsa = list(MolSurf.pyPEOE_VSA_(m))
        smr_vsa = list(MolSurf.pySMR_VSA_(m))
        logp_vsa = list(MolSurf.pySlogP_VSA_(m))

        mqns = rdMolDescriptors.MQNs_(m)

        each_desc = desc + graph_desc + estate_vsa + vsa_estate + peoe_vsa + smr_vsa + logp_vsa + mqns

        desc_2d.append(each_desc)

    desc_indices = ['num_aliphatic_carbocycles','num_alphatic_heterocycles',
                    'num_alphatic_ring','num_amidebond',
                    'num_aromatic_carbocycle','num_aromatic_heterocycle',
                    'num_aromatic_ring','num_atom_stereo_center',
                    'num_bridge_head_atom','num_HBA','num_HBD',
                    'num_heteroatoms','num_heterocycle',
                    'lipinski_HBA','lipinski_HBD','num_ring',
                    'num_rotatable_bond','num_saturated_carbocycle',
                    'num_saturated_heterocycle','num_saturated_ring',
                    'num_spiro_atoms','fraction_csp3','num_heavy_atom',
                    'num_NHOH','num_NO','mw','heavy_atom_mw',
                    'max_abs_estate_index','max_estate_index',
                    'min_abs_estate_index','min_estate_index',
                    'max_abs_partial_charge','max_partial_charge',
                    'min_abs_partial_charge','min_partial_charge',
                    'num_radical_electron','num_valence_electron',
                    'logP','MR','ASA','TPSA',

                    'balabanJ','bertzCT','Ipc','chi0','chi0n','chi0v',
                    'chi1','chi1n','chi1v','chi2n','chi2v','chi3n','chi3v',
                    'chi4n','chi4v','hallkierAlpha','kapp1','kappa2','kappa3']

    desc_indices_estate_vsa = []
    for i in range(1,12):
        desc_indices_estate_vsa.append('estate_vsa'+str(i))

    desc_indices_vsa_estate = []
    for i in range(1,11):
        desc_indices_vsa_estate.append('vsa_estate'+str(i))

    desc_indices_peoe_vsa = []
    for i in range(1,15): desc_indices_peoe_vsa.append('peoe_vsa'+str(i))

    desc_indices_smr_vsa = []
    for i in range(1,11): desc_indices_smr_vsa.append('smr_vsa'+str(i))

    desc_indices_logp_vsa = []
    for i in range(1,13): desc_indices_logp_vsa.append('logp_vsa'+str(i))

    desc_indices_mqns = ['mqns'+str(i) for i in range(42)]

    total_desc_indices = desc_indices + desc_indices_estate_vsa + desc_indices_vsa_estate + desc_indices_peoe_vsa + desc_indices_smr_vsa + desc_indices_logp_vsa + desc_indices_mqns

    return pd.DataFrame(desc_2d, columns=total_desc_indices)

def estateFP_calc(sd):
    return pd.DataFrame([FingerprintMol(m)[-1] for m in sd])

def autocorr2D_calc(sd):
    return pd.DataFrame([rdMolDescriptors.CalcAUTOCORR2D(m) for m in sd])

def maccsFP_calc(sd):
    maccfps = [MACCSkeys.GenMACCSKeys(m) for m in sd]
    maccfps_bit = fingerprint_bitgenerator(maccfps)
    return pd.DataFrame(maccfps_bit)

def avalonFP_calc(sd):
    avalonfps = [pyAvalonTools.GetAvalonFP(m) for m in sd]
    avalonfps_bit = fingerprint_bitgenerator(avalonfps)
    return pd.DataFrame(avalonfps_bit)

def avalonCountFP_calc(sd):
    avaloncountfps = [pyAvalonTools.GetAvalonCountFP(m) for m in sd]
    avaloncountfps_bit = fingerprint_bitgenerator(avaloncountfps)
    return pd.DataFrame(avaloncountfps_bit)

def layerFP_calc(sd):
    layerfps = [Chem.rdmolops.LayeredFingerprint(m) for m in sd]
    layerfps_bit = fingerprint_bitgenerator(layerfps)
    return pd.DataFrame(layerfps_bit)

def morganFP_calc(sd, level):
    circularfps = [AllChem.GetMorganFingerprintAsBitVect(m,level) for m in sd]
    circularfps_bit = fingerprint_bitgenerator(circularfps)
    return pd.DataFrame(circularfps_bit)

def atompairFP_calc(sd):
    atompairfp = [Pairs.GetAtomPairFingerprint(m) for m in sd]
    atompairfp_key = atompairfingerprint_keyextractor(atompairfp)
    atompairfp_bit = atompair_bitgenerator(atompairfp, atompairfp_key)
    return pd.DataFrame(atompairfp_bit,columns=list(range(len(atompairfp_bit[0]))))

def torsionFP_calc(sd):
    torsionfp = [Torsions.GetTopologicalTorsionFingerprint(m) for m in sd]
    torsionfp_key = atompairfingerprint_keyextractor(torsionfp)
    torsionfp_bit = atompair_bitgenerator(torsionfp, torsionfp_key)
    return pd.DataFrame(torsionfp_bit,columns=list(range(len(torsionfp_bit[0]))))

In [12]:
target = [train_df, test_df]

train_set = []
test_set = []

sets = [train_set, test_set]

for i, df in enumerate(target):
    df_id = df[['data_source', 'smi']]
    smi = df['smi']
    sd = [Chem.MolFromSmiles(m) for m in smi]  # Convert 'Smiles' to RDKit Mol objects
    y = df['value']  # pMIC

    desc2d = description2D_calc(sd)
    sets[i].append(pd.concat([df_id, y, desc2d], axis=1))

    estate = estateFP_calc(sd)
    sets[i].append(pd.concat([df_id, y, estate], axis=1))

    autocorr = autocorr2D_calc(sd)
    sets[i].append(pd.concat([df_id, y, autocorr], axis=1))

    maccs = maccsFP_calc(sd)
    sets[i].append(pd.concat([df_id, y, maccs], axis=1))

    avalon = avalonFP_calc(sd)
    sets[i].append(pd.concat([df_id, y, avalon], axis=1))

    avalon_count = avalonCountFP_calc(sd)
    sets[i].append(pd.concat([df_id, y, avalon_count], axis=1))

    layer = layerFP_calc(sd)
    sets[i].append(pd.concat([df_id, y, layer], axis=1))

    morgan2 = morganFP_calc(sd, 2)
    sets[i].append(pd.concat([df_id, y, morgan2], axis=1))

    morgan3 = morganFP_calc(sd, 3)
    sets[i].append(pd.concat([df_id, y, morgan3], axis=1))

    morgan4 = morganFP_calc(sd, 4)
    sets[i].append(pd.concat([df_id, y, morgan4], axis=1))


In [13]:
desc = ["desc2d","estateFP","autocorr2D","maccsFP","avalonFP","avalonCountFP","layerFP","morgan2","morgan3","morgan4"]

In [14]:
len(desc)

10

In [15]:
# import numpy as np
# from sklearn.metrics import mean_squared_error as MSE, r2_score
# from scipy import stats
# import xgboost


# for i in range(len(desc)):
#     X_train = train_set[i].drop(['data_source', 'smi', 'value'], axis=1)
#     y_train = train_set[i]['value']

#     X_test = test_set[i].drop(['data_source', 'smi', 'value'], axis=1)
#     y_test = test_set[i]['value']

#     print(f"* Descriptor Name : {desc[i]}")
#     print("  Training...")

#     xgb_model = xgboost.XGBRegressor(objective="reg:squarederror", n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
#                                      colsample_bytree=1, max_depth=7, seed=123)

#     xgb_model.fit(X_train, y_train)

#     # 모델 예측
#     pred = xgb_model.predict(X_test)

#     # 평가 메트릭 계산
#     r_sq = r2_score(y_test, pred)
#     rmse = np.sqrt(MSE(y_test, pred))
#     sp_r = stats.spearmanr(y_test, pred).statistic

#     print(" > Evaluation Metrics")
#     print(f"   R square : {r_sq}")
#     print(f"   RMSE : {rmse}")
#     print(f"   Spearman R : {sp_r}")
#     print("\n")


In [19]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from scipy import stats
from sklearn.model_selection import RandomizedSearchCV

# 모델과 그리드 정의
xgb_model = XGBRegressor(objective='reg:squarederror')

param_grids = {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.01, 0.05, 0.1],
        'gamma': [0, 0.1, 0.2],
        'subsample': [0.6, 0.75, 0.9],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 5, 7]
}


for i in range(len(desc)):
    X_train = train_set[i].drop(['data_source', 'smi', 'value'], axis=1)
    y_train = train_set[i]['value']

    X_test = test_set[i].drop(['data_source', 'smi', 'value'], axis=1)
    y_test = test_set[i]['value']

    print(f"* Descriptor Name: {desc[i]}")
    # grid_search = GridSearchCV(xgb_model, param_grids, cv=3, scoring='r2', n_jobs=-1, verbose=1)
    # grid_search.fit(X_train, y_train)
    # best_model = grid_search.best_estimator_
    # print(f"    Best Parameters: {grid_search.best_params_}")
    random_search = RandomizedSearchCV(xgb_model, param_grids, n_iter=100, cv=3, scoring='r2', n_jobs=-1, verbose=1, random_state=42)
    random_search.fit(X_train, y_train)
    best_model = random_search.best_estimator_
    print(f"    Best Parameters: {random_search.best_params_}")

    # 테스트 데이터에 대한 예측 및 성능 평가
    y_pred = best_model.predict(X_test)
    r_sq = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    sp_r = stats.spearmanr(y_test, y_pred).statistic

    # 결과 출력
    print(" > Evaluation Metrics")
    print(f"   R square : {r_sq:.4f}")
    print(f"   RMSE : {rmse:.4f}")
    print(f"   Spearman R : {sp_r:.4f}")
    print("\n")

* Descriptor Name: desc2d
Fitting 3 folds for each of 100 candidates, totalling 300 fits
    Best Parameters: {'subsample': 0.75, 'n_estimators': 150, 'max_depth': 7, 'learning_rate': 0.05, 'gamma': 0.1, 'colsample_bytree': 0.6}
 > Evaluation Metrics
   R square : 0.6251
   RMSE : 0.5531
   Spearman R : 0.7454


* Descriptor Name: estateFP
Fitting 3 folds for each of 100 candidates, totalling 300 fits
    Best Parameters: {'subsample': 0.6, 'n_estimators': 150, 'max_depth': 7, 'learning_rate': 0.1, 'gamma': 0.2, 'colsample_bytree': 0.8}
 > Evaluation Metrics
   R square : 0.5800
   RMSE : 0.5853
   Spearman R : 0.7098


* Descriptor Name: autocorr2D
Fitting 3 folds for each of 100 candidates, totalling 300 fits
    Best Parameters: {'subsample': 0.9, 'n_estimators': 150, 'max_depth': 7, 'learning_rate': 0.05, 'gamma': 0.2, 'colsample_bytree': 0.6}
 > Evaluation Metrics
   R square : 0.5242
   RMSE : 0.6230
   Spearman R : 0.6899


* Descriptor Name: maccsFP
Fitting 3 folds for each of 